In [1]:
from latent_geometry.mapping.torch import TorchModelMapping
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=16, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
in_tensor = torch.randn(1, 1, 16, 16)
out_tensor = net(in_tensor)
print(out_tensor)

tensor([[-0.1363, -0.0705, -0.1520, -0.0391, -0.0083,  0.1858,  0.0491, -0.0223,
          0.0379, -0.0511]], grad_fn=<AddmmBackward0>)


In [7]:
torch_mapping = TorchModelMapping(net, in_shape=(1, 1, 16, 16), out_shape=(1, 10))

In [8]:
in_numpy = in_tensor.numpy().reshape(-1)

In [9]:
torch_mapping(in_numpy)

array([-0.1363181 , -0.07047263, -0.15197027, -0.03908941, -0.00825167,
        0.18581575,  0.04910522, -0.02229357,  0.03785518, -0.0510995 ],
      dtype=float32)

In [10]:
J = torch_mapping.jacobian(in_numpy)
print(f"{J.shape=}")
print(f"{in_tensor.shape=}, {out_tensor.shape=}")
print(f"{np.prod(in_tensor.shape)=}, {np.prod(out_tensor.shape)=}")
J

J.shape=(10, 256)
in_tensor.shape=torch.Size([1, 1, 16, 16]), out_tensor.shape=torch.Size([1, 10])
np.prod(in_tensor.shape)=256, np.prod(out_tensor.shape)=10


array([[ 9.4759409e-05,  3.8494088e-04,  4.1702203e-04, ...,
         1.1372542e-03,  4.0279570e-04, -5.1381538e-04],
       [ 1.0407324e-03,  5.3860695e-04, -4.7073042e-04, ...,
         1.2515394e-04,  2.4954611e-04,  1.0927916e-04],
       [-1.6571312e-05,  1.6938765e-04,  1.8167244e-04, ...,
        -5.7250607e-05, -3.3224107e-05,  5.2521325e-05],
       ...,
       [ 1.3113787e-04,  3.7811382e-04,  2.6676129e-04, ...,
         6.9689169e-04,  8.8560802e-04, -3.8007495e-04],
       [-3.7397273e-04, -3.2689999e-04, -1.7406637e-05, ...,
        -3.5424618e-04, -6.4327259e-04,  2.7079051e-04],
       [ 1.5824822e-04, -8.7308414e-05, -1.8827047e-04, ...,
        -1.3863041e-04,  2.8537124e-04, -8.1905768e-05]], dtype=float32)

In [11]:
H = torch_mapping.second_derivative(in_numpy)
print(f"{H.shape=}")
print(f"{in_tensor.shape=}, {out_tensor.shape=}")
print(f"{np.prod(in_tensor.shape)=}, {np.prod(out_tensor.shape)=}")
H

H.shape=(10, 256, 256)
in_tensor.shape=torch.Size([1, 1, 16, 16]), out_tensor.shape=torch.Size([1, 10])
np.prod(in_tensor.shape)=256, np.prod(out_tensor.shape)=10


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.